In [323]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from pyvis.network import Network as net

In [324]:
g = nx.read_gpickle("objects/graph-A_3_0")

In [325]:
print(nx.info(g))

DiGraph named 'Graph from soil food web adjacency matrix' with 24 nodes and 94 edges


In [ ]:
g.nodes(data=True)

NodeDataView({'11 Plant-feeding nematode': {'trophic_level': 1, 'Biomass': 479278.83999999997, 'Mass': 6.031837019417812, 'size': 10}, '31 Bacterivore nematode': {'trophic_level': 1, 'Biomass': 97189.95, 'Mass': 0.44778320101132774, 'size': 10}, '45 Substrate-ingesting earthworm': {'trophic_level': 1, 'Biomass': 20876194.67, 'Mass': 656677.5800017535, 'size': 10}, '23 Fungivore insect': {'trophic_level': 1, 'Biomass': 87314.16, 'Mass': 123.42029317311494, 'size': 10}, '24 Fungivore enchytraeid': {'trophic_level': 1, 'Biomass': 666025.6699999999, 'Mass': 469.51500281899325, 'size': 10}, '21 Fungivore nematode': {'trophic_level': 1, 'Biomass': 17869.440000000002, 'Mass': 0.060594173232088044, 'size': 10}, '81 Omnivore nematode': {'trophic_level': 2, 'Biomass': 301148.6, 'Mass': 1.6196323444430343, 'size': 10}, '13 Plant-feeding insect (springtail)': {'trophic_level': 1, 'Biomass': 12483.4, 'Mass': 8.82985650742963, 'size': 10}, '44 Substrate-inhabiting enchytraeid': {'trophic_level': 1, 

In [326]:
NxG = net('1300px', '1300px')
# populates the nodes and edges data structures
NxG.from_nx(g)

In [327]:
# Import trophic level as node attribute
trophic_levels = pd.read_csv("data/trophic_levels_1_0.csv", index_col=0, header=None, delimiter=";", names=["trophic_level"])
trophic_levels = trophic_levels.squeeze()

# Create dict from trophic level Series
troph_lev_dict = trophic_levels.to_dict()
troph_lev_dict

{'11 Plant-feeding nematode': 1,
 '31 Bacterivore nematode': 1,
 '45 Substrate-ingesting earthworm': 1,
 '23 Fungivore insect': 1,
 '24 Fungivore enchytraeid': 1,
 '21 Fungivore nematode': 1,
 '81 Omnivore nematode': 2,
 '13 Plant-feeding insect (springtail)': 1,
 '44 Substrate-inhabiting enchytraeid': 1,
 '51 Predating nematode (consuming nematodes)': 3,
 '72 Generalist mite': 3,
 '34 Bacterivore enchytraeid': 1,
 '82 Omnivore mite': 2,
 '12 Macrophytophage and panphytophage mite': 1,
 '36 Bacterivore amoebae': 1,
 '37 Bacterivore flagellates': 1,
 '#0 Plant autotrophs (roots)': 0,
 '41 "Passive lifestage, substrate-related nematode"': 1,
 '48 Primary (heterotrophic) producer bacteria': 0,
 '49 Primary (heterotrophic) producer fungi': 0,
 '22 Microphytophage mite (feeding on fungi)': 1,
 '62 Predatory mite (attacking arthropods)': 3,
 '52 Predatory mite (attacking nematodes)': 3,
 '92 Predatory mite (parasitizing mites and nematodes)': 3}

In [328]:
trophic_levels = list(troph_lev_dict.values())
colours = []
for _ in trophic_levels:
    if _ == 0:
        colours.append("#40E0D0")
    elif _ == 1:
        colours.append("#DFFF00")
    elif _ == 2:
        colours.append("#FFBF00")
    elif _ == 3:
        colours.append("#DE3163")

In [329]:
# Create a PyVis graph G
G = net('1300px', '1300px')
nodes = list(g.nodes()) 
G.add_nodes(nodes, color=colours)

### Create network divided by trophic level 0 as source

In [330]:
# list of primary producers
primaries = ["#0 Plant autotrophs (roots)", "48 Primary (heterotrophic) producer bacteria", "49 Primary (heterotrophic) producer fungi"]

#### 48 bacteria

In [401]:
# Keep only one primary producer in a new network
g48 = nx.read_gpickle("objects/graph-A_3_0")

# list of primary producers
primaries = ["#0 Plant autotrophs (roots)", "48 Primary (heterotrophic) producer bacteria", "49 Primary (heterotrophic) producer fungi"]

# selected primary porducer
primary = "48 Primary (heterotrophic) producer bacteria"

# define network nodes
for n in primaries:
    if n != primary:
        # remove from Nxgraph
        g48.remove_node(n) 
        
# remove from nodes list
nodes48 = list(g48.nodes())

# map trophic level to colours
colours = []
for n in nodes48:
    tl = troph_lev_dict[n]
    if tl == 0:
        colours.append("#40E0D0")
    elif tl == 1:
        colours.append("#DFFF00")
    elif tl == 2:
        colours.append("#FFBF00")
    elif tl == 3:
        colours.append("#DE3163")

In [402]:
# Remove nodes from trophic level 1 without link to trophic level 0
good_nodes48_2l = []
for n in nodes48:
    for source, target in list(g48.in_edges(n)):
        if g48.nodes[n]["trophic_level"] == 1 and source == primary:
            good_nodes48_2l.append(n)

# list of all nodes on trophic level 1
all_nodes48_2l = []
for n in list(g48.nodes()):
    if g48.nodes[n]["trophic_level"] == 1:
        all_nodes48_2l.append(n)


# make list of bad nodes:
bad_nodes48_2l = all_nodes48_2l.copy()
for _ in good_nodes48_2l:
    bad_nodes48_2l.remove(_)

g48.remove_nodes_from(bad_nodes48_2l)

In [403]:
nx.info(g48)

"DiGraph named 'Graph from soil food web adjacency matrix' with 15 nodes and 52 edges"

In [404]:
# Create a list (of tuples) of edges of a network for the chose primary producer as source
edgesG48 = []
for source, target in list(g48.edges()):
    if g48.nodes[target]["trophic_level"] >= g48.nodes[source]["trophic_level"]:
        edgesG48.append((source, target))

In [405]:
# Create a PyVis graph for this primary producer
G48 = net('1300px', '1300px')
G48.add_nodes(nodes48, color=colours)

# Add edges to PyVis graph
for source, target in edgesG48:
    G48.add_edge(source, target)

In [399]:
# Edges take on the color of the node they are coming from.
G48.inherit_edge_colors(status=True)

In [406]:
G48.show_buttons(filter_=True)
G48.show('SFW-48_1.html')

In [337]:
# G48.set_options("""
# var options = {
#   "nodes": {
#     "font": {
#       "size": 15,
#       "strokeWidth": 5
#     }
#   },
#   "edges": {
#     "arrows": {
#       "middle": {
#         "enabled": true,
#         "scaleFactor": 0.8
#       }
#     },
#     "color": {
#       "inherit": true
#     },
#     "smooth": false
#   },
#   "physics": {
#     "enabled": false,
#     "minVelocity": 0.75
#   }
# }
# """) 
# G48.show('SFW-48_1.html')